In [ ]:
import glob
import os

import pandas as pd
from pandas.api.types import is_string_dtype
from dask import compute
from dask.delayed import delayed
import dask.dataframe as dd
import zipfile

from config import NOAA_GOLD_PATH, ERCOT_GOLD_PATH, STATION, AREA, TRAINING_GOLD_PATH

# NOAA Observations

In [ ]:
df_noaa = dd.read_parquet(NOAA_GOLD_PATH)
df_noaa.head()

In [ ]:
df_noaa_filter = df_noaa["station"]==STATION
df_noaa_prep = (
    df_noaa
    .where(df_noaa_filter)
    .drop(columns=["raw"])
    .set_index("datetime")
)
df_noaa_prep.head()

# ERCOT Loads

In [ ]:
df_ercot = dd.read_parquet(ERCOT_GOLD_PATH)
df_ercot.head()

In [ ]:
df_ercot_filter = ~(df_ercot["Hour Beginning"].isna()) # (df_ercot["area"]==AREA) & 

df_ercot_prep = (
    df_ercot
    .where(~df_ercot["Hour Beginning"].isna())
    .where(df_ercot["area"]=="COAST")
    .dropna()
    .rename(columns={"Hour Beginning": "datetime"})
    .drop(columns=["Hour Ending", "Hour Ending Raw", "year"])
    .set_index("datetime")
)
df_ercot_prep.compute()

# Training Dataset

In [ ]:
df_training = df_noaa_prep.join(df_ercot_prep, on="datetime", how="inner").reset_index()
df_training.columns

In [ ]:
df_training.to_parquet(TRAINING_GOLD_PATH, partition_on=["station", "year"])

In [ ]:
dd.read_parquet(TRAINING_GOLD_PATH).head()

Previous: [Transform Ercot Load Data](transform_ercot_loads.ipynb)
Next: [Visualize Training DataSet](viz_training_dataset.ipynb)